In [1]:
# import the necessary packages
from imutils.perspective import four_point_transform
from imutils import contours
from math import ceil
from matplotlib import pyplot as plt
import numpy as np
import imutils
import cv2
import os
import time

In [2]:
# import sys
# sys.path.append("E:\Python 3.7\Lib\site-packages")
# print(sys.path)

In [3]:
def get_xval(answer): # dạng của s (image, [x, y, w, h])
    return answer[1][0]

def is_rect(cnt):
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
    #if our approximated contour has four points,
    # then we can assume we have found the paper
    if len(approx) == 4:
        return True
    return False

def show_img(img):
    cv2.imshow('img', img)
    cv2.waitKey(0)
    
def show_resized_img(img, width, height):
    resized_image = cv2.resize(img, (width, height))
    cv2.imshow('img', resized_image)
    cv2.waitKey(0)

In [4]:
def thresh_img(image):

    # ADAPTIVE_THRESH_MEAN_C
    # ADAPTIVE_THRESH_GAUSSIAN_C

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#     blurred = cv2.medianBlur(gray, 5)
#     thresh = cv2.Canny(blurred, 75, 200)

#     thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 3)
    
#     thresh = cv2.dilate(thresh,None,iterations = 1)
#     thresh = cv2.erode(thresh,None,iterations = 1)
    
    sx = cv2.Sobel(thresh, cv2.CV_32F, 1, 0)
    sy = cv2.Sobel(thresh, cv2.CV_32F, 0, 1)
    m = cv2.magnitude(sx, sy)
    thresh = cv2.normalize(m, None, 0.0, 255.0, cv2.NORM_MINMAX, cv2.CV_8U)
    

#     show_img(gray)
#     show_img(blurred)
#     show_img(thresh)
    
    return thresh

In [5]:
def retr_codetest_id(code_test_img):
    code_id = []
    
    code_test_img = cv2.cvtColor(code_test_img, cv2.COLOR_BGR2GRAY)
    code_test_img = cv2.threshold(code_test_img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    code_test_cols = np.array_split(code_test_img, 3, 1)

    for i, cols in enumerate(code_test_cols):
#         show_img(cols)

        max_nonz = 0
        filled_idx = -1
        
        code_id_bubble = np.array_split(cols, 10, 0)
        for j, b in enumerate(code_id_bubble):
#             show_resized_img(b, 250, 250)
#             show_img(b)
            curr_nonz = cv2.countNonZero(b)
            if curr_nonz > max_nonz:
                filled_idx = j
                max_nonz = curr_nonz    
        code_id.append((i + 1, filled_idx))
    return code_id

In [6]:
def retr_student_id(student_id_img):
    student_id = []
    
    student_id_img = cv2.cvtColor(student_id_img, cv2.COLOR_BGR2GRAY)
    student_id_img = cv2.threshold(student_id_img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    student_id_cols = np.array_split(student_id_img, 6, 1)
    
    for i, cols in enumerate(student_id_cols):
#         show_img(cols)

        max_nonz = 0
        filled_idx = -1
        
        student_id_bubble = np.array_split(cols, 10, 0)
        for j, b in enumerate(student_id_bubble):
#             show_resized_img(b, 250, 250)
#             show_img(b)
            curr_nonz = cv2.countNonZero(b)
            if curr_nonz > max_nonz:
                filled_idx = j
                max_nonz = curr_nonz    
        student_id.append((i + 1, filled_idx))
    return student_id

In [7]:
def get_choice(argument):
    switcher = {
        0: "A",
        1: "B",
        2: "C",
        3: "D",
    }
    return switcher.get(argument, "blank")

In [8]:
def get_answer(answer_list):
    key_gen = []
    
    for i, ans in enumerate(answer_list):
        ans = cv2.cvtColor(ans, cv2.COLOR_BGR2GRAY)
        ans = cv2.threshold(ans, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        
#         show_img(ans)
        bubble_row = np.array_split(ans, 5, 1)
        bubble_row.pop(0)
        
        max_nonz = 0
        choice = ''
        filled_idx = -1
        for j, b in enumerate(bubble_row):
            curr_nonz = cv2.countNonZero(b)
#             show_img(b)
            if curr_nonz > max_nonz:
                filled_idx = j
                max_nonz = curr_nonz
                
        choice = get_choice(filled_idx)
        key_gen.append((i + 1, choice))
    return key_gen

In [9]:
def retr_choice(sorted_ans_blocks):
    #collect answer
    box_distance = 10

    answer_list = []
    key_gen = []
    
    for ans_block in sorted_ans_blocks:
        img = np.array(ans_block[0]) #take each answer block image

        #take each answer box
        box_img_rows = np.array_split(img, 6, 0)
        
        for tmp_img in box_img_rows: 
            img = tmp_img[box_distance:tmp_img.shape[0]-box_distance, :]
            
#             show_img(img)
            answer_rows = np.array_split(img, 5, 0)
            for j in answer_rows:
                answer_list.append(j)
#                 show_img(j)
                
    key_gen = get_answer(answer_list)
    return key_gen

In [10]:
def retrInfo(org_img):
    # find contours in the edge map, then initialize
    # the contour that corresponds to the document
    result = []
    image = org_img.copy()
    
    thresh = thresh_img(image)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = imutils.grab_contours(cnts)

    answer_block = []

    # ensure that at least one contour was found
    if len(cnts) > 0:
        # sort the contours according to their size in
        # descending order

        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        
#         cv2.drawContours(img, cnts, -1, (0, 255, 0), 3)
#         show_img(img)
        
    
        answer_block_coor = cnts[0:4]

        student_id_coor = cnts[6]
        code_test_coor = cnts[7]

        # loop over the sorted contours
        for c in answer_block_coor:
            # approximate the contour

            x,y,w,h = cv2.boundingRect(c)
            answer_block.append((image[y:y + h, x:x + w], [x, y, w, h]))
#             show_img(image[y:y + h, x:x + w])

            #collect answer block
        sorted_ans_blocks = sorted(answer_block, key=get_xval)
        
        key_gen = retr_choice(sorted_ans_blocks)
        
        for k in key_gen:
            print(k)
        
        x,y,w,h = cv2.boundingRect(student_id_coor)
        student_id_img = image[y:y + h, x:x + w]
        
#         show_img(student_id_img)
        student_id_block = retr_student_id(student_id_img)

        print('---student id------')
        for num in student_id_block:
            print(num)

            
        x,y,w,h = cv2.boundingRect(code_test_coor)
        code_test_img = image[y:y + h, x:x + w]
#         show_img(code_test_img)
        
        code_id = retr_codetest_id(code_test_img)

        print('---code test id------')
        for num in code_id:
            print(num)
    
    result.append(key_gen)
    result.append(student_id_block)
    result.append(code_id)
    return result

In [11]:
# start_time = time.time()

In [12]:
# for i in range(200):
#     filename = 'answer_gen/ans_' + str(i + 1) + '.png'
#     image = cv2.imread(filename)
    
#     print('---------------Scanning img ' + str(i + 1) + '--------------------------')
#     retrInfo(image)


In [13]:
# print("--- %s seconds ---" % (time.time() - start_time))

# API 

In [26]:
from flask import Flask, request, redirect, jsonify
import urllib
import numpy as np

from functools import wraps
from flask_restful import Resource, Api, reqparse
import werkzeug
from werkzeug.utils import secure_filename
import pandas as pd
import ast

app = Flask(__name__)
api = Api(app)

In [27]:
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # return the image
    return image

In [28]:
# app.url_map

In [29]:
class UploadImage(Resource):
    def post(self):
        result = []
        url = request.args.get('url')
#         print(url)
        image = url_to_image(url)
        result = retrInfo(image)
        return result

In [30]:
api.add_resource(UploadImage, '/api')

In [31]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Dec/2021 18:25:18] "POST /api?url=https://res.cloudinary.com/dunbjnt9i/image/upload/v1640082054/test-img/ans_95_nfolac.png HTTP/1.1" 200 -


(1, 'B')
(2, 'D')
(3, 'A')
(4, 'B')
(5, 'B')
(6, 'A')
(7, 'C')
(8, 'B')
(9, 'C')
(10, 'A')
(11, 'D')
(12, 'B')
(13, 'B')
(14, 'D')
(15, 'B')
(16, 'B')
(17, 'A')
(18, 'A')
(19, 'D')
(20, 'B')
(21, 'C')
(22, 'A')
(23, 'C')
(24, 'C')
(25, 'B')
(26, 'C')
(27, 'B')
(28, 'C')
(29, 'A')
(30, 'D')
(31, 'A')
(32, 'B')
(33, 'A')
(34, 'D')
(35, 'D')
(36, 'B')
(37, 'A')
(38, 'B')
(39, 'C')
(40, 'D')
(41, 'B')
(42, 'C')
(43, 'C')
(44, 'B')
(45, 'B')
(46, 'C')
(47, 'D')
(48, 'D')
(49, 'C')
(50, 'B')
(51, 'B')
(52, 'C')
(53, 'B')
(54, 'A')
(55, 'D')
(56, 'C')
(57, 'C')
(58, 'C')
(59, 'D')
(60, 'B')
(61, 'A')
(62, 'B')
(63, 'D')
(64, 'A')
(65, 'B')
(66, 'C')
(67, 'C')
(68, 'A')
(69, 'C')
(70, 'D')
(71, 'B')
(72, 'C')
(73, 'B')
(74, 'D')
(75, 'D')
(76, 'B')
(77, 'C')
(78, 'C')
(79, 'B')
(80, 'A')
(81, 'D')
(82, 'D')
(83, 'B')
(84, 'B')
(85, 'C')
(86, 'B')
(87, 'B')
(88, 'B')
(89, 'A')
(90, 'A')
(91, 'D')
(92, 'C')
(93, 'D')
(94, 'B')
(95, 'D')
(96, 'B')
(97, 'A')
(98, 'A')
(99, 'C')
(100, 'D')
(101, 'A

In [32]:
urllib.__version__

AttributeError: module 'urllib' has no attribute '__version__'